In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 
####################you will need to change some paths here!#####################
#list of input files
#list of input files


In [ ]:
df = pd.read_excel('./../data/sst_error.xlsx')
ds = df.to_xarray().rename({'SST':'sst'})
ds = ds.swap_dims({'index':'sst'}).drop('index')
ds

In [ ]:
ds.std_11GHz.plot()
ds.std_7GHz.plot()

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=20)
client = Client(cluster)
cluster

In [ ]:
%%time
file_location = 's3://mur-sst/zarr'
ds_sst = xr.open_zarr(fsspec.get_mapper(file_location, anon=True),consolidated=True)
ds_sst

In [ ]:
sst_climatology = sst_timeseries.groupby('time.month').mean()
sst_climatology_yr = sst_timeseries.mean({'time'})

In [ ]:
map7 = ds.std_7GHz.interp(lat=ds_sst.lat,lon=ds_sst.lon)
map11 = ds.std_11GHz.interp(lat=ds_sst.lat,lon=ds_sst.lon)
mapd = map11 - map7